Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 8.2 MB 40.9 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6464ad017960baf7b945143484dd582d3e0777c25da2b0fdfe20edb3aa725293
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [2]:
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [3]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 23.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_excel('/content/drive/MyDrive/отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
max_words = 20000
max_len = 150
num_classes = 5

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [9]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [10]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [11]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:7]

['приложение', 'очень', 'удобно', 'работать', 'удобный', 'вс', 'спасибо']

In [12]:
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [13]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [14]:
data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']], 
    dtype=np.int32)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [16]:
X_train.shape

(14461, 150)

In [17]:
le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

le.classes_

array([1, 2, 3, 4, 5])

### 2.1 предобученные эмбединги

In [22]:
embeddings_index = {}
with open('/content/drive/MyDrive/model.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        emb_idxs[word] = coefs

print("Found %s word vectors." % len(emb_idxs))

Found 249947 word vectors.


In [23]:
embedding_dim = 200

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [24]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    weights=[embedding_matrix], 
                    output_dim=embedding_dim, 
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [25]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 29s 1s/step - loss: 1.5973 - accuracy: 0.6807 - val_loss: 1.5834 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 26s 993ms/step - loss: 1.5716 - accuracy: 0.7051 - val_loss: 1.5578 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 25s 977ms/step - loss: 1.5466 - accuracy: 0.7051 - val_loss: 1.5329 - val_accuracy: 0.7091
Epoch 4/10
26/26 [==============================] - 25s 980ms/step - loss: 1.5222 - accuracy: 0.7051 - val_loss: 1.5089 - val_accuracy: 0.7091
Epoch 5/10
26/26 [==============================] - 25s 979ms/step - loss: 1.4986 - accuracy: 0.7051 - val_loss: 1.4853 - val_accuracy: 0.7091
Epoch 6/10
26/26 [==============================] - 26s 983ms/step - loss: 1.4755 - accuracy: 0.7051 - val_loss: 1.4625 - val_accuracy: 0.7091
Epoch 7/10
26/26 [==============================] - 25s 977ms/step - loss: 1.4532 - accuracy: 0.7051 - val_loss: 1.4403 - val_accuracy: 0.7091
Ep

In [27]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 208ms/step - loss: 1.3789 - accuracy: 0.7073
Test score: 1.378862977027893
Test accuracy: 0.7073249220848083


### 2.2 эмбединги по умолчанию

In [28]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [29]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 16s 587ms/step - loss: 1.1181 - accuracy: 0.6977 - val_loss: 0.9009 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 15s 575ms/step - loss: 0.7874 - accuracy: 0.7245 - val_loss: 0.6975 - val_accuracy: 0.7560
Epoch 3/10
26/26 [==============================] - 15s 576ms/step - loss: 0.6441 - accuracy: 0.7769 - val_loss: 0.6613 - val_accuracy: 0.7713
Epoch 4/10
26/26 [==============================] - 15s 576ms/step - loss: 0.5985 - accuracy: 0.7894 - val_loss: 0.6494 - val_accuracy: 0.7740
Epoch 5/10
26/26 [==============================] - 15s 575ms/step - loss: 0.5653 - accuracy: 0.7947 - val_loss: 0.6448 - val_accuracy: 0.7740
Epoch 6/10
26/26 [==============================] - 15s 577ms/step - loss: 0.5324 - accuracy: 0.8053 - val_loss: 0.6416 - val_accuracy: 0.7789
Epoch 7/10
26/26 [==============================] - 19s 719ms/step - loss: 0.5015 - accuracy: 0.8193 - val_loss: 0.6401 - val_accuracy: 0.7809

In [31]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 140ms/step - loss: 0.6806 - accuracy: 0.7701
Test score: 0.6805828213691711
Test accuracy: 0.7700871229171753


Точность на предобученных эмбедингах ниже. 